In [ ]:
!pip install pyedflib
!pip install numpy
!pip install pandas
!pip install pickle
!pip install scipy

In [ ]:
import pyedflib
import numpy as np
import pandas as pd
from numpy import *
import pandas
import pickle
import scipy.io as sio

# Multiple channels to single channel

In [ ]:
# Allows plots to appear directly in the notebook.
%matplotlib inline
#data_path='D:/EEG/EDF/Crumlin/try_separate/identify_AGE/6/train/'
# Load the edf file
f = pyedflib.EdfReader("6-22.edf")
n = f.signals_in_file
signal_labels = f.getSignalLabels()
sigbufs = np.zeros((n, f.getNSamples()[0]))

for i in np.arange(n):
    sigbufs[i, :] = f.readSignal(i)

# and load the data into a DataFrame
df_signals = pd.DataFrame(sigbufs)
df_signals = df_signals.transpose()
df_signals.columns = signal_labels

In [ ]:
print('The sample frequency of your data is: ', f.getSampleFrequency(0))

## Now convert the edf format to csv file

In [ ]:
df_signals.columns=['EEG1','EEG2']
signal_channel=df_signals.EEG2
path='D:/Desktop/Deployment-flask-master/'
signal_channel=pd.DataFrame(signal_channel)
signal_channel.columns=['EEG']
signal_channel=signal_channel[14400000:16200000]
signal_channel.to_csv(path+'EEG_6_22_8-9h.csv', index=False)

# Bio-channel 

In [ ]:
class ChannelHeader:
    def __init__(self,seq,name):
        self.seq = seq
        self.name = name
        self.offset_in_chunk = -1
        self.sample_rate = -1
        #chunk_size means number of elements in one chunk
        self.chunk_size = -1

In [ ]:
#coding=utf-8 
# Input
#   edf_filename: full path name of EDF file
# Return
#   data_map: key=channel_name, value=data list
#   cHdrList: header info of each channel
def readRawDataFromEdf(edf_filename):
    with io.open(edf_filename, 'rb') as f:
        cHdrList = readHeader(f)
        #DEBUG
        chunk_size = 0
        for cHdr in cHdrList:
            print ("%s, sample_rate=%d" % (cHdr.name, cHdr.sample_rate))
            chunk_size = chunk_size + cHdr.sample_rate
        print ("total bytes in one chunk: %d" % ((2*chunk_size)))
        data_map = readData(f, cHdrList)
  
    return data_map,cHdrList

#read header information from EDF file stream
#INPUT f: handle to EDF file, caller must gurantee the position is set to START of the file
#RETURN cHdrList: a list, each element represent a channel
def readHeader(f):
    f.seek(243,io.SEEK_CUR)
    chunk_time_len = int(f.read(8))
    #print chunk_time_len
    num_of_channels = int(f.read(4))
    #print num_of_channels
    
    #read num_of_channels*16 byte for names
    cHdrList = []
    for cid in range(num_of_channels):
        channel_name = str(f.read(16)).strip()
        cHdrList.append(ChannelHeader(cid, channel_name))
    
    #Skip data until sample rate
    f.seek(num_of_channels*200, io.SEEK_CUR)
    
    #read sample rate of each channel
    offset = 0
    for cid in range(num_of_channels):
        chunk_size = int(f.read(8))
        cHdrList[cid].offset_in_chunk = offset
        cHdrList[cid].chunk_size = chunk_size
        cHdrList[cid].sample_rate = chunk_size / chunk_time_len

    #skip reserved field
    f.seek(num_of_channels*32 + 1, io.SEEK_CUR)
  
    return cHdrList

#read data from EDF file, each data is a 16 bits integer with sign
#INPUT f: handle to EDF file, caller must gurantee the position is set to START + (num_of_channel + 1 )*256, i.e. the start position of data section
#RETURN data_map: key=channel_name, value=data list
def readData(f, cHdrList):
    #prepare the result data
    data_map = {}
    for cHdr in cHdrList:
        data_map[cHdr.name]=[]
  
    #read each channel 
    eof = False
    chunk_cnt = 0
    while True:
        for cHdr in cHdrList:
        #read data for only one channel in one chunk
            for dataIdx in range(cHdr.chunk_size):
                data_array = f.read(2)
            #DEBUG
            #print data_array.encode('hex')
                if (len(data_array) == 0):
                    eof = True
                    break
                #unpack using little-endian
                data = struct.unpack('<h', data_array)[0]
                data_map[cHdr.name].append(data)
      
            if eof:
                break
    
        chunk_cnt = chunk_cnt +1
        if eof:    
            print ("Chunk_count = %d" % (chunk_cnt))
            break
  
    return data_map  

def writeDataIntoDisk(filename, dataList):
    with open(filename,'w') as f:
        for data in dataList:
            f.write("%d," % data)

def ensureUtf(s):
    try:
        if type(s) == unicode:
            return s.encode('utf8', 'ignore')
    except: 
        return str(s)

In [ ]:
filename =ensureUtf("D:/desk/EEG/mouse/Rogerio_data/Lan-data/P24_b_M1.edf")
data_map,cHdrList = readRawDataFromEdf(filename)

## Note: sample rate is your sample frequency

In [ ]:
data=pd.DataFrame.from_dict(data_map, orient='index')

In [ ]:
df=data.transpose()
df.columns = ['C3','C4','CZ','F3','F4','F7','F8','FZ','FP1','FP2','FPZ','O1','O2','P3','P4','PZ','T3','T4','T5','T6','AUX1',
              'AUX2','AUX3','AUX4','AUX5','AUX6','AUX7','AUX8','PG1' ,'PG2' ,'A1','A2','EDF Annotations']

In [ ]:
T6_P4=df['T6']-df['P4']
O2_P4=df['O2']-df['P4']
F7_F3=df['F7']-df['F3']
C3_F3=df['C3']-df['F3']
F8_F4=df['F8']-df['F4']
C4_F4=df['C4']-df['F4']
T5_P3=df['T5']-df['P3']
O1_P3=df['O1']-df['P3']

In [ ]:
frames = [T6_P4,O2_P4,F7_F3,C3_F3,F8_F4,C4_F4,T5_P3,O1_P3]
df_bipolar = pd.concat(frames,axis=1)
df_bipolar.columns = ['T6_P4','O2_P4','F7_F3','C3_F3','F8_F4','C4_F4','T5_P3','O1_P3']

In [ ]:
import pickle
#读取文件中的内容。注意和通常读取数据的区别之处
df=open('D:/desk/EEG/Rogerio_mice/rogerio_updated_0926/Rogerio_updated_P24.pickle','rb')#注意此处是rb
#此处使用的是load(目标文件)
data=pickle.load(df)
df.close()

In [ ]:
df1=data['C3_F3']
df1.columns=['EEG']

In [ ]:
path='D:/desk/EDF/Deployment-flask-master/'
f.to_csv(path+'type2_EEG.csv', index=False)

# Matlab to csv

In [ ]:
import scipy.io as sio
data = sio.loadmat('D:/desk/EDF/data/sp1s_aa.mat')

In [ ]:
new_data=data['x_train']

In [ ]:
new_data.shape[2]

In [ ]:
new_data=new_data.T

In [ ]:
new_data

In [ ]:
import numpy
new_data=numpy.reshape(new_data, (new_data.shape[0]*new_data.shape[2], new_data.shape[1]))

In [ ]:
import pandas as pd
new_data=pd.DataFrame(new_data)
new_data

In [ ]:
new_data.columns=['F3','F1','Fz','F2','F4','FC5','FC3','FC1','FCz','FC2','FC4','FC6','C5','C3',
                  'C1','Cz','C2','C4','C6','CP5','CP3','CP1','CPz','CP2','CP4','CP6','O1','O2']
new_data

# txt to csv 

In [ ]:
df = pd.read_fwf('D:/desk/EDF/data/sp1s_aa_train.txt', header=None)

In [ ]:
df

# pickle to csv

In [ ]:
import pickle
#读取文件中的内容。注意和通常读取数据的区别之处
df=open('F:/Mouse_EEG/Type_II_Rogerio/M/M4-7_P21-28/M4_7/M7_O1_P3.pickle','rb')#注意此处是rb
#此处使用的是load(目标文件)
data=pickle.load(df)
df.close()

In [ ]:
data

In [ ]:
df1=data['EEG2']

In [ ]:
df1.to_csv('mouse1_typeII.csv', index=False)